## **gold-yahoo-financial-news**

This notebook builds the **`yahoo_financial_news`** Delta table by combining income statement, cash flow, balance sheet, and company info data from the Silver layer. It unifies financial records into a single dataset and performs an upsert to keep the table updated. Error handling ensures smooth processing and traceability of issues.


#### **Importing necessary libraries**

In [10]:
from pyspark.sql.functions import *
from delta.tables import DeltaTable
import datetime
import traceback

StatementMeta(, 197655e9-4fc8-4297-97b7-89d36dd4e19e, 12, Finished, Available, Finished)

#### **Variables**

In [11]:
layer_name = "silver"
table_name = "yahoo_stock_news"

StatementMeta(, 197655e9-4fc8-4297-97b7-89d36dd4e19e, 13, Finished, Available, Finished)

#### **Step 2: Reading the Stock News and Information Data**

In [12]:
com_news_df = spark.read.format('delta').load(f"Files/{layer_name}/comp_news_data/")

com_info_df = spark.read.format('delta').load(f"Files/{layer_name}/comp_info_data/")

StatementMeta(, 197655e9-4fc8-4297-97b7-89d36dd4e19e, 14, Finished, Available, Finished)

#### **Step 3: Joining Financial News and Company Information**

In [13]:
output_df = com_news_df\
                .join(com_info_df, com_info_df.company_name == com_news_df.company)\
                .select(
                        col('short_name').alias('company_name'),
                        col('company_image').alias('company_logo'),
                        col('industry'),
                        col('sector'),
                        col('title'),
                        col('Date'),
                        col('news_agency'),
                        col('content_type'),
                        col('news_url'),
                        col('news_summary')
                )


StatementMeta(, 197655e9-4fc8-4297-97b7-89d36dd4e19e, 15, Finished, Available, Finished)

#### **Step 4: Storing the transformed data into a Delta table in the Gold layer**

In [14]:
try:
    if DeltaTable.isDeltaTable(spark, table_name):
        delta_table = DeltaTable.forPath(spark, table_name)
               
        delta_table.alias("target").merge(
            output_df.alias("source"),
                (
                    (col("target.company_name") == col("source.company_name")) &
                    (col("target.Date") == col("source.Date"))
                )
            ).whenMatchedUpdateAll() \
             .whenNotMatchedInsertAll() \
             .execute()

    else:
        output_df.write.format("delta").mode("overwrite").saveAsTable(table_name)

except Exception as e:
    print("An error occurred during the processing:")
    traceback.print_exc()

StatementMeta(, 197655e9-4fc8-4297-97b7-89d36dd4e19e, 16, Finished, Available, Finished)